In [ ]:
# @title
import pickle
import pandas as pd
import glob
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
experiment_name = "2024.01.04-01_T5_base_zero-shot_NEW"

In [ ]:
# @title
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

output_data_path = base_path + "/../Flan-T5/data/output/" + experiment_name
log_filename = base_path + "/../Flan-T5/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# @title
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
# model_id = "google/flan-t5-small" # 60M
model_id = "google/flan-t5-base" # 220M
# model_id = "google/flan-t5-large" # 800M
# model_id = "google/flan-t5-xl" # 3B
# model_id = "google/flan-t5-xxl" # 11B

In [ ]:
# @title
# nbit = 4
# # llamasize = '7B'
# llamasize = '13B'
# temp = 0.1
# outdir = 'data/output/2023.11.13-1_T5_run_completo-large___TEST/'
# outdir = 'data/output/2023.11.22-6_T5_run_completo-base_zero-shot-REBOOT/'
# outfilename = outdir+f'sdg_classification_T5_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
outfilename = output_data_path+f'/sdg_classification_T5_1020-texts.xlsx'

outfilename

'/content/gdrive/MyDrive/Colab/SDG/data/../Flan-T5/data/output/2024.01.04-01_T5_base_zero-shot_NEW/sdg_classification_T5_1020-texts.xlsx'

In [ ]:
data = pd.read_excel(input_data_path)

In [ ]:
# @title
data_sel = data
# data_sel = data.sample(10)

In [ ]:
data_sel = data

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)
    if res[3] == "<pad> SDG_99</s>":
      res = (res[0], res[1], res[2], "<pad> SDG_0</s>")
    results.append(res)

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'id', 'true_label', 'predicted_label'])
results_processed

,cnt,id,true_label,predicted_label
0,323,4198,5,<pad> SDG_8</s>
1,325,2883,5,<pad> SDG_4</s>
2,326,4899,5,<pad> SDG_5</s>
3,327,4761,5,<pad> SDG_5</s>
4,330,486,5,<pad> SDG_5</s>
...,...,...,...,...
1015,316,4649,5,<pad> SDG_8</s>
1016,318,5833,5,<pad> SDG_4</s>
1017,319,2793,5,<pad> SDG_16</s>
1018,321,4040,5,<pad> SDG_5</s>


In [ ]:
# results_processed['label'] = results_processed['predicted_label'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['predicted_label'].str.replace('<pad> ', '')
results_processed['label'] = results_processed['label'].str.replace('</s>', '')

In [ ]:
results_processed.groupby('label').count()

,cnt,id,true_label,predicted_label
label,,,,
SDG_0,402,402,402,402
SDG_1,16,16,16,16
SDG_10,12,12,12,12
SDG_11,26,26,26,26
SDG_12,8,8,8,8
SDG_13,9,9,9,9
SDG_14,41,41,41,41
SDG_15,79,79,79,79
SDG_16,99,99,99,99


In [ ]:
results_processed['label'] = results_processed['label'].str.replace("_", " ")
results_processed['label'] = results_processed['label'].str.extract(r'SDG (\d+)')

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,<pad> SDG_0</s>,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,<pad> SDG_0</s>,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,<pad> SDG_0</s>,0


In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
# final.to_excel(outfilename)

In [ ]:
final = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.10      0.68      0.18        60
           1       0.75      0.20      0.32        60
           2       0.90      0.30      0.45        60
           3       0.47      0.25      0.33        60
           4       0.79      0.82      0.80        60
           5       1.00      0.37      0.54        60
           6       0.95      0.33      0.49        60
           7       0.89      0.57      0.69        60
           8       0.35      0.70      0.47        60
           9       0.62      0.13      0.22        60
          10       0.83      0.17      0.28        60
          11       0.73      0.32      0.44        60
          12       0.38      0.05      0.09        60
          13       1.00      0.15      0.26        60
          14       0.78      0.53      0.63        60
          15       0.62      0.82      0.71        60
          16       0.39      0.65      0.49        60

    accuracy    

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [ ]:
report_df

,precision,recall,f1-score,support
0,10.199005,68.333333,17.748918,60.000000
1,75.000000,20.000000,31.578947,60.000000
2,90.000000,30.000000,45.000000,60.000000
3,46.875000,25.000000,32.608696,60.000000
4,79.032258,81.666667,80.327869,60.000000
5,100.000000,36.666667,53.658537,60.000000
6,95.238095,33.333333,49.382716,60.000000
7,89.473684,56.666667,69.387755,60.000000
8,35.000000,70.000000,46.666667,60.000000
9,61.538462,13.333333,21.917808,60.000000


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

67.98439981026004 41.37254901960784 41.37254901960784 43.41639764688623


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

68.0 41.4 41.4 43.4
